# Setup of particle mass flow rate in Fluent DEM

When setting up a DPM model with DEM interactions enabled, one is often found with the need to pre-initialize the system with a particle filling. This is not supported by Ansys Fluent and some trick can be used as proposed [here](https://www.youtube.com/watch?v=snKyEOxwnys). The idea is to release all the particles at a very short interval when simulation starts. This notebook provides a means of computing the mass flow rate compatible with this release time and particle distribution.

We start by importing required external packages.

In [1]:
import numpy as np

In Fluent built-in DEM particles are assumed to be spheric, thus we introduce the following functions.

In [2]:
def sphere_volume(R: float) -> float:
    """ Compute the volume of an sphere. """
    return (4 / 3) * np.pi * pow(R, 3)


def sphere_mass(rho: float, R: float) -> float:
    """ Returns the mass of a single particle. """
    return rho * sphere_volume(R)

Assuming a known radii distribution, the mass flow rate of particles can be computed by integrating (*i.e* adding up) the mass of particles and dividing the result by the release time used in the start of unsteady simulation.

In [3]:
def dist_mass_flow(rho: float, radii: list[float], inj_time: float) -> float:
    """ Mass flow of particles to be injected. """
    return sum(sphere_mass(rho, radii)) / inj_time

Now let's get to a practical example. Assume you want to inject during a short time of `inj_time = 1.0e-08` seconds a number of `num_part = 100` particles of specific weight `rho = 2320.0` in the domain. Particle size distribution is known and of constant radius `R_min = 0.01` meter (notice that this is compatible with `linear` option in Ansys Fluent, you can use other distributions to compute radii of particles). Since the software expects a mass flow rate input, this can be computed as follows.

In [4]:
inj_time = 1.0e-08

num_part = 100
rho = 2320.0

R_min = 0.01
R_max = R_min

radii = np.linspace(R_min, R_max, num_part)
mass = dist_mass_flow(rho, radii, inj_time)

mass

97179932.75104448

That's a pretty high value because of the short release time. It is recommended to run a single fluid time-step (which must be larger than the selected value for release time) and check by the end of integration if (approximately) the right number of particles is found inside the domain.